In [ ]:
import sys
sys.path.append('../')
from Backend.RAICAT.utils import check_dns_measurements,compute_average_by_country

res=check_dns_measurements("2022-05-01")
res2=check_dns_measurements("2022-05-02")


In [ ]:
import concurrent.futures
from Backend.RAICAT.utils import check_dns_measurements,compute_average_by_country
def compute_dns_measurements_for_two_dates(date1, date2):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Submit the functions for execution
        future1 = executor.submit(lambda: check_dns_measurements(date1))
        future2 = executor.submit(lambda: check_dns_measurements(date2))

        # Wait until both functions are completed
        concurrent.futures.wait([future1, future2])
        result1 = future1.result()
        result2 = future2.result()
    print()
compute_dns_measurements_for_two_dates("2022-05-01","2022-05-02")

In [7]:
from pprint import pprint


def compute_average(x):
    if len(x) == 0:
        return 0
    return round(sum(x) / len(x), 1)





frontend_aggregated_results = prepare_results_for_frontend(res2)
pprint(frontend_aggregated_results)

{'average': 146.0,
 'data': {'ALB': 93.0,
          'AND': 21.4,
          'ARE': 38.6,
          'ARG': 452.6,
          'ARM': 74.5,
          'ATA': 977.8,
          'AUS': 73.3,
          'AUT': 34.9,
          'AZE': 60.8,
          'BEL': 21.0,
          'BGD': 50.3,
          'BGR': 42.6,
          'BHR': 9.8,
          'BIH': 255.8,
          'BLM': 117.9,
          'BLR': 40.2,
          'BLZ': 820.6,
          'BOL': 342.2,
          'BRA': 158.6,
          'BRN': 80.4,
          'CAN': 55.9,
          'CHE': 23.8,
          'CHL': 153.7,
          'CHN': 158.2,
          'COD': 69.8,
          'COK': 142.9,
          'COL': 79.4,
          'CRI': 308.0,
          'CUW': 162.5,
          'CYM': 88.7,
          'CYP': 210.4,
          'CZE': 26.8,
          'DEU': 33.4,
          'DJI': 80.1,
          'DNK': 27.7,
          'DOM': 98.3,
          'DZA': 117.1,
          'ECU': 253.7,
          'EGY': 80.1,
          'ESP': 29.0,
          'EST': 80.3,
          'FIN': 48.4,
 

In [ ]:
min([value for key,value in res.items()])

In [ ]:

from Backend.RAICAT.utils import compute_average_by_country
res
compute_average_by_country(res)

In [ ]:

from pprint import pprint
from ipwhois.experimental import bulk_lookup_rdap,get_bulk_asn_whois
ip_list = ['74.125.225.229', '2001:4860:4860::8888', '62.239.237.1', '2a00:2381:ffff::1', '210.107.73.73',
        #    '2001:240:10c:1::ca20:9d1d', '200.57.141.161', '2801:10:c000::', '196.11.240.215', '2001:43f8:7b0::', '133.1.2.5', '115.1.2.3'
           ]
# obj = IPWhois('["8.8.4.4", "1.1.1.1", "2c0f:fb50:4003::"]')
results= get_bulk_asn_whois(addresses=ip_list)
# results, stats = bulk_lookup_rdap(addresses=ip_list)
pprint(results)